In [134]:
import tweepy

API_KEY = "HZh8MFUydAVaIN50fYh2w5iVr"
API_SECRET = "BjkcY9H2tZhTHH2gDiyeByRJaBWExXxW523ZXvhy5jyh0mTlLn"
ACCESS_TOKEN = "301590896-xTlLyxVVYURtMRFDyhBa4BHEOEAHheCa9iEdicQv"
ACCESS_TOKEN_SECRET = "90soM5pLMXfuWkP1BIGVEDZFrBWkearvAiEeJ72JbW5rc"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [135]:
def tweetSearch(query, limit = 100, language = "fr", date_since="2019-12-05", date_until="2019-12-06"):
    text = ""
    for tweet in tweepy.Cursor(api.search, q=query, lang=language, since=date_since, until=date_until).items(limit):
        text += tweet.text.lower() + ";;"
    return text

In [136]:
greve = tweetSearch("#Grève")

In [137]:
greve = greve.split(';;')
greve = set(greve)
greve = [i for i in greve if i] 

In [138]:
greve[:10]

["rt @caathaarsis: une lacrymo tombe dans le système de filtration d'air d'une salle de sport de wazemmes. plusieurs gamines gazées. un flic…",
 "rt @f3aquitaine: #grève &gt; à quoi faut-il s'attendre demain vendredi 6 décembre en aquitaine ? \nhttps://t.co/yzrkqqq8iz",
 'rt @triboque: la meilleure chose qui pourrait sortir du mouvement contre la réforme des retraites serait que les moins de 45ans prennent co…',
 'rt @stylos_les: on manifeste sous toutes les latitudes contre la réforme des retraites. #tahiti \n\n#stylosrouges #5decembre #5decembre2019 #…',
 'rt @aquatennens: j’étais invité ce soir à intervenir en duplex sur @france2tv pour l’émission #valp afin d’interpeller le gouvernement dont…',
 'rt @stylos_les: les enseignants défilent à #grenoble contre la réforme des retraites !\n\n#stylosrouges #5decembre #5decembre2019 #greve5dece…',
 'rt @jlmelenchon: énorme succès de la mobilisation partout en france ! \n\nça sent le roussi pour #macron !\n\n#greve5decembre #grevedu5décembre

In [147]:
def clean_up(x):
    import re
    a=[]
    for s in x:
        s=s.replace("""'""",' ')
        s= re.sub(r'http\S+|(www\.[^\s]+)|(@\S+)',' ',s)
        s=re.sub('[\d]',' ',s)
        s= re.sub('[^\w\s]',' ',s)
        s= re.sub('\s\s+',' ',s)
        s=s.lower().strip()
        a.append(s)
    return a
def stem_and_lemmatize(l):
    from nltk import stem
    stemmer = stem.PorterStemmer()
    o=[]
    for b in l:
        o.append(stemmer.stem(b))
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    p=[]
    for a in o:
        p.append(lemmatizer.lemmatize(a))
    return p
def stop_word(l):
    from nltk.corpus import stopwords 
    from nltk.tokenize import word_tokenize 
    stop_words = stopwords.words('french')
    stop_words.append('rt')
    filtered_sentence=[]
    for a in l:
        word_tokens = word_tokenize(a)
        filtered_sentence.append([w for w in word_tokens if not w in stop_words])
    return [' '.join(x) for x in filtered_sentence]

In [148]:
greve_clean = clean_up(greve)

In [149]:
greve_clean[:5]

['rt une lacrymo tombe dans le système de filtration d air d une salle de sport de wazemmes plusieurs gamines gazées un flic',
 'rt grève gt à quoi faut il s attendre demain vendredi décembre en aquitaine',
 'rt la meilleure chose qui pourrait sortir du mouvement contre la réforme des retraites serait que les moins de ans prennent co',
 'rt on manifeste sous toutes les latitudes contre la réforme des retraites tahiti stylosrouges decembre decembre',
 'rt j étais invité ce soir à intervenir en duplex sur pour l émission valp afin d interpeller le gouvernement dont']

In [150]:
greve_clean2 = stop_word(greve_clean)
greve_clean2[:5]

['lacrymo tombe système filtration air salle sport wazemmes plusieurs gamines gazées flic',
 'grève gt quoi faut attendre demain vendredi décembre aquitaine',
 'meilleure chose pourrait sortir mouvement contre réforme retraites moins ans prennent co',
 'manifeste sous toutes latitudes contre réforme retraites tahiti stylosrouges decembre decembre',
 'invité soir intervenir duplex émission valp afin interpeller gouvernement dont']

In [151]:
import pandas as pd
greve_tweets = pd.DataFrame(columns=['tweet','positif/negatif'])

In [152]:
greve_tweets['tweet']=greve_clean2

In [153]:
greve_tweets.to_csv('greve_tweets.csv')